# Protocol Controller Example

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. Che

In [12]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [13]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

In [14]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'state': 'active', 'accept': 'manual', 'created_at': '2020-09-07 09:50:13.759252Z', 'request_id': '8d3b7081-c3e0-4387-8235-ed4565e8bc7a', 'invitation_mode': 'once', 'my_did': '8tzPhqTQQoJhEW9y7g3G7b', 'updated_at': '2020-09-07 09:50:23.905062Z', 'initiator': 'external', 'their_label': 'Bob', 'their_did': 'JkKDqYAkeKGaj6UBWM6N3B', 'invitation_key': 'HWejSmnF6U34ow5JBJwrsPypdntTqE8Hs9kZTCu5rZh6', 'routing_state': 'none', 'connection_id': '7bf9a9be-efba-43cb-bbd1-6bf31035f6ea'}]
Connection : {'state': 'active', 'accept': 'manual', 'created_at': '2020-09-07 09:50:13.759252Z', 'request_id': '8d3b7081-c3e0-4387-8235-ed4565e8bc7a', 'invitation_mode': 'once', 'my_did': '8tzPhqTQQoJhEW9y7g3G7b', 'updated_at': '2020-09-07 09:50:23.905062Z', 'initiator': 'external', 'their_label': 'Bob', 'their_did': 'JkKDqYAkeKGaj6UBWM6N3B', 'invitation_key': 'HWejSmnF6U34ow5JBJwrsPypdntTqE8Hs9kZTCu5rZh6', 'routing_state': 'none', 'connection_id': '7bf9a9be-efba-43cb-bbd1-6bf31035f6ea'}
Active Conne

In [15]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)

message_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([message_listener], defaults=True)

Task exception was never retrieved
future: <Task finished coro=<AriesAgentController.listen_webhooks() done, defined at /aries-basic-controller/aries_basic_controller/aries_controller.py:64> exception=OSError(98, "error while attempting to bind on address ('0.0.0.0', 8022): address already in use")>
Traceback (most recent call last):
  File "/aries-basic-controller/aries_basic_controller/aries_controller.py", line 70, in listen_webhooks
    await self.webhook_site.start()
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_runner.py", line 104, in start
    reuse_port=self._reuse_port)
  File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 1385, in create_server
    % (sa, err.strerror.lower())) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8022): address already in use


In [16]:
from aiohttp import FormData


In [17]:
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')

In [18]:
response = await om_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


Attachment- Alice-> Bob
{'thread_id': 'a475877a-aa89-45fd-b94c-e531ef37f241'}


In [68]:
response = await om_controller.terminate()
print(response)


None
